In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


import scipy.stats as stats 
from scipy.stats import chi2_contingency, boxcox

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm 

In [6]:
df = pd.read_csv('Maternal-risk-processed.csv')

# PREDICCIÓN DE RIESGO MATERNAL CON ALGORITMO GRADIENT BOOST CLASSIFIER

In [9]:
x = df.drop('RiskLevel', axis=1)
y = df['RiskLevel']

In [11]:
#Separamos datos entre muestra de entrenamiento y muestra de test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=101, stratify=y)

**Optimización de los Hiperparámetros con GridSearchCV**
<UL>
    <li>n_estimators: número deárboles en el modelo. Más árboles mejoran el rendimiento, pwro incrementa el tiempo de entrenamiento.</li>
    <li>learning_rate: tasa de aprendizaje. Reduce la contribución de cada árbol, valores más pequeños requieren más árboles pero el modelo es más robusto.</li>
    <li>max_depth: profundidad máxima de los árboles.Limitar la profundidad reduce el sobreajuste.</li>
    <li>loss: para la clasificación, función de pérdida a optimizar. Suelen ser <code>log_loss</code> (para clasificación o <code>exponential</code>.</li>
    <li>min_samples_split: número mínimo de muestras necesarias para dividir un nodo. Controla la creación de nodos de un árbol. Valores más altos hacen que el árbol sea  más restrictivo.</li>
    <li>min_samples_leaf: número mínimo de muestras en una hoja.</li>
    <li>max_leaf_nodes: número máximo de hojas en el árbol, limita la complejidad del modelo.</li>
    <li>max_features: número máximo de características consideradas para dividir un nodo. Hay varias opciones: auto o sqrt (raíz cuadrada del númerototal de características), log2 (logaritmo base 2 del número total de características), o None (todas las características). </li>
    <li>subsample: fracción de datos usada para entrenar cada árbol, los valores menores a 1 introducen aleatoriedad y ayudan a reducir el sobreajuste.</li>
    <li>criterion: función que mide la división; opciones: friedman_mse: es el predeterminado, squared_error: error cuadrático medio, o mae: error absoluto medio (más robusto a outliers). </li>
</UL>

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

#Modelo básico
gbc = GradientBoostingClassifier(random_state=42, 
                                n_estimators=500, learning_rate=0.01,   max_depth =5, 
                                min_samples_split=7, min_samples_leaf=5, subsample= 0.6, 
                                max_features=None)
gbc.fit(X_train, y_train)

#predicciones
y_pred = gbc.predict(X_test)

#Evaluación del Modelo
print("Accuraccy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación")
print(classification_report(y_test, y_pred))

Accuraccy: 0.7205882352941176
Reporte de clasificación
              precision    recall  f1-score   support

           0       0.72      0.91      0.81        70
           1       0.50      0.22      0.30        32
           2       0.82      0.79      0.81        34

    accuracy                           0.72       136
   macro avg       0.68      0.64      0.64       136
weighted avg       0.69      0.72      0.69       136



In [18]:
param_grid = {
    'n_estimators': [100, 200, 500, 700], 
    'learning_rate':[0.001,0.01, 0.05, 0.1],
    'max_depth':[2,3,4,5,7], 
    'min_samples_split':[2,5,10,15,20],
    'min_samples_leaf':[1,2,4,6,8], 
    'subsample':[0.5, 0.6, 0.8, 1.0], 
    'max_features':['sqrt', 'log2', None],
    'criterion': ['friedman_mse', 'squarred_error', 'mae']
}

grid_search = GridSearchCV(
    estimator=gbc, 
    param_grid=param_grid,
    scoring='accuracy',
    cv=5, 
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
print("Mejores hiperparámetros encontrados:", grid_search.best_params_)

Fitting 5 folds for each of 72000 candidates, totalling 360000 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

#Modelo básico
gbc = GradientBoostingClassifier(random_state=42, 
                                n_estimators=200, class_weight = 'balanced', 
                                learning_rate=0.01,   max_depth =5, 
                                min_samples_split=7, min_samples_leaf=5, subsample= 0.6, 
                                max_features=None)
gbc.fit(X_train, y_train)

#predicciones
y_pred = gbc.predict(X_test)

#Evaluación del Modelo
print("Accuraccy:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación")
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(gbc, X_train, y_train, cv=5)

# Calcular medias
train_mean = train_scores.mean(axis=1)
test_mean = test_scores.mean(axis=1)

# Graficar
plt.plot(train_sizes, train_mean, label="Training Score")
plt.plot(train_sizes, test_mean, label="Validation Score")
plt.xlabel("Training Size")
plt.ylabel("Accuracy")
plt.legend()
plt.show()